In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [ ]:
from case_code import CodeDownloader, get_metadata, CodeLoader, transform, Case
from utils.model_selector import get_chat_model

In [ ]:
# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()
print(f"{'Total files downloaded:':<24} {downloaded_count}")

metadata = get_metadata()
print(f"{'Total cases in metadata:':<24} {len(metadata)}")

In [ ]:
import pandas as pd

# Get the chat model
model = get_chat_model()

# Transform the downloaded code to case
loader = CodeLoader()
skipped = await transform(model=model, loader=loader, total=downloaded_count)

data = [(key.value, len(value)) for key, value in skipped.items()]
df = pd.DataFrame(data, columns=["Error", "Count"])
print(df)

In [ ]:
from case_code.code_transformer import TransformError

print(f"Parsed error cases: {skipped[TransformError.ParseError]}")

In [ ]:
from case import get_retriever

retriever = get_retriever()

In [ ]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [ ]:
question = "Stake 0.01 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Stake 3 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())